## Llama 2 Embeddings

### imports

In [16]:
from tqdm import tqdm
import json
import pandas as pd
import sys
sys.path.append('..')

from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from db_connect import db_get_df, db_save_df

### manuelle Erstellung

In [ ]:


model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', trust_remote_code = True)
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

In [ ]:
df = db_get_df("transcript_segments")

In [ ]:
len(df["segment_text"][1300:1600].to_list())

In [ ]:
input_ids = tokenizer(
    df["segment_text"].to_list(), 
    return_tensors = 'pt',
    padding = True
)

In [ ]:
v = model.encode(input_ids).detach().numpy()
v.shape

In [ ]:
df["embedding_json"] = [json.dumps(model.encode(chunk_text).detach().numpy()) for chunk_text in tqdm(input_ids)]


In [ ]:
input_ids["input_ids"][0]

In [ ]:
len(v[0])

### Vereinigen der Tables

In [5]:
df = db_get_df()
len(df)

3114

In [14]:
table_id = list("bcdyefghij")
all_df = db_get_df("transcript_segments_llama_2_a")
for id in table_id:
    df_temp = db_get_df(f"transcript_segments_llama_2_{id}")
    # print(len(df_temp))
    print(df_temp.head(1).iloc[0, 0])
    all_df = pd.concat([all_df, df_temp])


-0.005324719939380884
-0.037108391523361206
0.0047004432417452335
0.023355837911367416
-0.001974386628717184
0.03071478009223938
-0.006729209329932928
0.003163792658597231
0.005811205133795738
0.02588091418147087


In [17]:
db_save_df(all_df, "transcript_segments_llama_2_all")

In [21]:
all_df = db_get_df("transcript_segments_llama_2_all")

In [22]:
df = db_get_df()

In [23]:
json_strings = [json.dumps(row.tolist()) for index, row in all_df.iterrows()]
df["embedding_json"] = json_strings

In [24]:
df.iloc[3]

filename                     jonathan-swift-gullivers-reisen-2.json
segment_text       Seile herbei, Flöcke und Leitern. Sie binden ...
start                                                         34.96
end                                                           42.28
segment_id                                                        3
embedding_json    [0.015561574138700962, 0.026827841997146606, -...
Name: 3, dtype: object

In [25]:
db_save_df(df, "transcript_segments_llama_2")